# Custom Dataset을 이용한 Hugging Face BERT model Fine Tuning

- NAVER Movie review dataset을 이용하여 transformers BERT model을 fine tuning  

- Pytorch 와 Trainer를 이용한 Fine Tuning (Pytorch version이 Tensorflow 보다 안정적)

In [1]:
!pip install transformers[torch]
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 32.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 35.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 30.6 MB/s eta 0:00:00


In [2]:
from transformers import BertTokenizer
from transformers import BertForSequenceClassification, Trainer, TrainingArguments
import torch.nn.functional as F
import tensorflow as tf
import pandas as pd

In [3]:
DATA_TRAIN_PATH = tf.keras.utils.get_file("ratings_train.txt",
            "https://raw.githubusercontent.com/junhyeok8696/NLP/main/naver_movie/ratings_train.txt")
DATA_TEST_PATH = tf.keras.utils.get_file("ratings_test.txt",
            "https://raw.githubusercontent.com/junhyeok8696/NLP/main/naver_movie/ratings_test.txt")

4893335/4893335 [==============================] - 0s 0us/step


### Train Set

In [4]:
train_data = pd.read_csv(DATA_TRAIN_PATH, delimiter='\t')
print(train_data.shape)
train_data.head()

(150000, 3)


,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [5]:
train_data.isnull().sum()

id          0
document    5
label       0
dtype: int64

In [6]:
train_data.dropna(inplace=True)
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 149995 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        149995 non-null  int64 
 1   document  149995 non-null  object
 2   label     149995 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 4.6+ MB


### Test Set

In [7]:
test_data = pd.read_csv(DATA_TEST_PATH, delimiter='\t')
print(test_data.shape)
test_data.head()

(50000, 3)


,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0


In [8]:
test_data.isnull().sum()

id          0
document    3
label       0
dtype: int64

In [9]:
test_data.dropna(inplace=True)
test_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49997 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        49997 non-null  int64 
 1   document  49997 non-null  object
 2   label     49997 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.5+ MB


- 훈련 시간 단축을 위해 1/10 의 data 만 sampling - 6분 소요

In [10]:
df_train = train_data.sample(n = 15000, random_state = 1)
df_test = test_data.sample(n = 5000, random_state = 1)
print(df_train.shape)
print(df_test.shape)

(15000, 3)
(5000, 3)


In [11]:
df_train['label'].value_counts()

0    7524
1    7476
Name: label, dtype: int64

In [12]:
X_train = df_train['document'].values.tolist()
y_train = df_train['label'].values.tolist()

X_test = df_test['document'].values.tolist()
y_test = df_test['label'].values.tolist()

In [13]:
X_train[:5]

['(평점조절용 1) 애니인데 분위기가 좀 음산? 그로테스크하고, 캐릭터들이 무민가족 빼고 인간인지 뭔지 다른 종족은 기괴해요. 괴팍한, 우울한, 종말론적인 캐릭터들이 많이 나오고 무민가족 외에 다른 캐릭터간에는 정상적인 의사소통(대화)아닌 일방적..',
 '성우님들의 열연이 마음에 들었습니다',
 '무지막지하게 지루함. 배우들이 아까움. 내용도 앞뒤없고 별로 들어오는 내용도 없고 답답함. 이야기를 좀더 쉽게 보여주던지.... 이도 저도 아니고 내가 수목장이 되어서 나무가 되어버린 느낌... 답답하다..',
 '대통령마누라의 어이없는 죽음, 알아듣기 힘든 한국말 차라리 영어만 하등가, 북한놈이라는 세끼가 씨오브재팬이라질 않나, 동해에 칠함대가 철수하면 한국 망한다고 미친 존나 한국을 OO으로 생각하는 감독, 백악관이 이렇게 쉽게 털리면 누가 테러안하겠냐 앙?',
 '모이따구 영화를?']

## pre-trained bert model 호출
### tokenizer 호출
- 토큰화 처리를 합니다. bert 다국어 version 용의 pre-trained tokenizer 를 불러옵니다.

In [14]:
# .from_pretrained('') -> 사용할 모델 토크나이저 불러오기
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

pre-trained tokenizer 를 이용하여 train set 과 test set 을 token 화 합니다.

- Input IDs : 토큰 인덱스, 모델에서 입력으로 사용할 시퀀스를 구축하는 토큰의 숫자 표현
- Token Type IDs : 한 쌍의 문장 또는 질문 답변에 대한 분류 시 사용  
- attention mask : `1`은 주목해야 하는 값을 나타내고 `0`은 패딩된 값을 나타냅니다.  
```
[CLS] SEQUENCE_A [SEP] SEQUENCE_B [SEP]
ex) [CLS] HuggingFace is based in NYC [SEP] Where is HuggingFace based? [SEP]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1]
```

In [15]:
train_encodings = tokenizer(X_train, truncation=True, padding=True)
test_encodings = tokenizer(X_test, truncation=True, padding=True)

In [16]:
train_encodings.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [17]:
print(train_encodings['input_ids'][0])
print(train_encodings['attention_mask'][0])
print(train_encodings['token_type_ids'][0])

[101, 113, 9926, 34907, 20626, 58931, 24974, 122, 114, 9532, 25503, 12030, 28911, 9367, 19855, 47869, 9682, 9634, 21386, 136, 8924, 11261, 119351, 12605, 20308, 12453, 117, 9792, 73352, 21876, 20173, 9294, 36553, 11287, 52560, 9391, 11664, 9640, 18784, 12030, 12508, 9304, 12508, 19709, 9684, 52560, 10892, 8932, 118651, 14523, 48549, 119, 8905, 119377, 11102, 117, 9604, 78123, 11102, 117, 9684, 89523, 42769, 15387, 9792, 73352, 21876, 20173, 47058, 8982, 28188, 11664, 9294, 36553, 11287, 52560, 9597, 10530, 19709, 9792, 73352, 21876, 100698, 11018, 9670, 14871, 15387, 9637, 12945, 22333, 43022, 113, 9069, 18227, 114, 63783, 9641, 42337, 14801, 119, 119, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

### Convert encodings to Tensors

- 레이블과 인코딩을 Dataset 개체로 변환합니다. Pytorch를 이용합니다.  

- PyTorch에서 이것은 `torch.utils.data.Dataset` 객체를 하고 `__len__` 및 `__getitem__`을 구현하여 수행됩니다.

- TensorFlow에서는 입력 인코딩과 레이블을 `from_tensor_slices` 생성자 메서드에 전달합니다. (불안정)

In [18]:
import torch

class IMDbDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

train_dataset = IMDbDataset(train_encodings, y_train)
test_dataset = IMDbDataset(test_encodings, y_test)

이제 데이터 세트가 준비되었으므로 🤗 `Trainer` 또는 기본 PyTorch/TensorFlow를 사용하여 모델을 미세 조정할 수 있습니다. [training](https://huggingface.co/transformers/training.html)을 참조하세요.

- Training warmup steps :  

    - 이는 일반적으로 설정된 수의 훈련 단계(워밍업 단계)에 대해 매우 낮은 학습률을 사용한다는 것을 의미합니다. 워밍업 단계 후에 "일반" 학습률 또는 학습률 스케줄러를 사용합니다. 또한 워밍업 단계 수에 따라 학습률을 점진적으로 높일 수 있습니다.

- weight_decay : 가중치 감쇠. L2 regularization

In [19]:
training_args = TrainingArguments(
    output_dir='./results',               # output 저장 directory
    num_train_epochs=2,              # total number of training epochs
    per_device_train_batch_size=8,  # batch size per device during training -> 병렬처리 시
    per_device_eval_batch_size=16,   # batch size per device during evaluation -> 병렬처리 시
    warmup_steps=500,                # number of warmup steps for learning rate scheduler -> warmup_steps 지정 수 까지 lr을 높게 후 decay 시킴
    weight_decay=0.01,               # weight decay 강도
    logging_dir='./logs',            # log 저장 directory
    logging_steps=10,
)

### model Train

In [20]:
import time

model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased')

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

s = time.time()

trainer.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss
10,0.726100
20,0.695100
30,0.709700
40,0.692700
50,0.692800
60,0.668800
70,0.692200
80,0.692700
90,0.660800
100,0.690600


TrainOutput(global_step=3750, training_loss=0.5367001820246379, metrics={'train_runtime': 920.3031, 'train_samples_per_second': 32.598, 'train_steps_per_second': 4.075, 'total_flos': 1896249598200000.0, 'train_loss': 0.5367001820246379, 'epoch': 2.0})

In [21]:
print("경과 시간 : {:.2f}분".format((time.time() - s)/60))

경과 시간 : 17.63분


In [22]:
trainer.evaluate(test_dataset)

{'eval_loss': 0.47213906049728394,
 'eval_runtime': 33.4564,
 'eval_samples_per_second': 149.448,
 'eval_steps_per_second': 9.355,
 'epoch': 2.0}

In [23]:
prediction = trainer.predict(test_dataset)

fine-tuned model 은 logit 을 return

In [24]:
trainer.model.classifier

Linear(in_features=768, out_features=2, bias=True)

In [25]:
y_logit = torch.tensor(prediction[0])
y_logit[:10]

tensor([[-0.2251,  0.3387],
        [ 1.4189, -1.2427],
        [ 1.4217, -1.2499],
        [ 1.4217, -1.2728],
        [-0.9800,  1.3213],
        [ 1.4224, -1.2699],
        [ 0.1144, -0.2335],
        [ 1.4231, -1.2659],
        [-1.4281,  1.7348],
        [ 1.4057, -1.2155]])

In [26]:
y_pred = F.softmax(y_logit, dim=-1).argmax(axis=1).numpy()
print(list(y_pred[:30]))
print(y_test[:30])

[1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1]
[0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1]


In [29]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

print(accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.7928
[[1967  528]
 [ 508 1997]]
              precision    recall  f1-score   support

           0       0.79      0.79      0.79      2495
           1       0.79      0.80      0.79      2505

    accuracy                           0.79      5000
   macro avg       0.79      0.79      0.79      5000
weighted avg       0.79      0.79      0.79      5000



In [33]:
x = '돈주고 보기에는 아까운 영화 ㅠㅠ...'
y = '정말 재미있는 영화'

x_tokenized = tokenizer([x], truncation=True, padding=True)
y_tokenized = tokenizer([y], truncation=True, padding=True)

x_pred = trainer.predict(IMDbDataset(x_tokenized))
y_pred = trainer.predict(IMDbDataset(y_tokenized))

x_logit = torch.tensor(x_pred[0])
y_logit = torch.tensor(y_pred[0])

x_result = F.softmax(x_logit, dim=-1).argmax(1).numpy()
y_result = F.softmax(y_logit, dim=-1).argmax(1).numpy()

print("긍정" if x_result == 1 else "부정")
print("긍정" if y_result == 1 else "부정")

부정
긍정
